In [74]:
!which python

/Users/apabook/Desktop/ds-deployment/venv/bin/python


In [75]:
import requests
import json
import pandas as pd
from urllib.error import HTTPError, URLError

# Complete dataspace flow

## Consts


In [76]:
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

## Wallets setup

In [77]:
url = data_space_provider + "/api/v1/wallet/onboard"
response = requests.request("POST", url)
response

<Response [201]>

In [78]:
url = data_space_consumer + "/api/v1/wallet/onboard"
response = requests.request("POST", url)
response

<Response [201]>

## Participants onboarding

### Current Participants information

In [79]:
url = data_space_provider + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0",
    "participant_slug": "Myself",
    "participant_type": "Provider",
    "base_url": "http://127.0.0.1:1203",
    "token": null,
    "token_actions": null,
    "saved_at": "2025-07-15T09:58:44.439351",
    "last_interaction": "2025-07-15T10:51:34.192783",
    "is_me": true
}


In [80]:
url = data_space_consumer + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
    "participant_slug": "Myself",
    "participant_type": "Consumer",
    "base_url": "http://127.0.0.1:1100",
    "token": null,
    "token_actions": null,
    "saved_at": "2025-07-15T09:58:47.486006",
    "last_interaction": "2025-07-15T10:51:36.941641",
    "is_me": true
}


### Consumer registering in Provider via Verifiable presentation

In [81]:
url = data_space_provider + "/api/v1/.well-known/did.json"
response = requests.request("GET", url)
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["id"]
data_space_provider_participant_id

'did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0'

In [82]:
url = data_space_consumer + "/api/v1/auth/manual/ssi"
payload = {
    "url": data_space_provider + "/api/v1/access",
    "slug": "provider",
    "id": data_space_provider_participant_id,
    "actions": "talk"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_text = response.text
response_as_text

'openid4vp://authorize?response_type=vp_token&client_id=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fverify%2FarHHjcZyVfYB&response_mode=direct_post&presentation_definition_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fpd%2FarHHjcZyVfYB&client_id_scheme=redirect_uri&nonce=oMkqKxPH2KEh&response_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fverify%2FarHHjcZyVfYB'

### Provider and Consumer ids

In [83]:
url = data_space_provider + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Consumer":
        data_space_consumer_participant_id = participant["participant_id"]
        data_space_consumer_participant_token = participant["token"]
        break

print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
        "participant_slug": "\"rainbow_consumer\"",
        "participant_type": "Consumer",
        "base_url": "http://127.0.0.1:1100",
        "token": "qIIAdMoQf-4E6sg74h-kkMGpqhahhzmPyhWL-wIqEVA",
        "token_actions": "talk",
        "saved_at": "2025-07-15T10:12:03.987594",
        "last_interaction": "2025-07-15T10:12:03.987596",
        "is_me": false
    },
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0",
        "participant_slug": "Myself",
        "participant_type": "Provider",
        "base_url": "http://127.0.0.1:1203",
        "token": null,
        "token_actions": null,
        "sa

In [84]:
url = data_space_consumer + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Provider":
        data_space_provider_participant_id = participant["participant_id"]
        data_space_provider_participant_token = participant["token"]
        break

print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0",
        "participant_slug": "provider",
        "participant_type": "Provider",
        "base_url": "http://127.0.0.1:1200",
        "token": "qIIAdMoQf-4E6sg74h-kkMGpqhahhzmPyhWL-wIqEVA",
        "token_actions": "talk",
        "saved_at": "2025-07-15T10:12:04.011377",
        "last_interaction": "2025-07-15T10:12:04.011380",
        "is_me": false
    },
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
        "participant_slug": "Myself",
        "participant_type": "Consumer",
        "base_url": "http://127.0.0.1:1100",
        "token": null,
        "token_actions": null,
        "saved_at": "20

## Fetch Catalog data from Datahub

In [85]:
url = data_space_provider + "/api/v1/datahub/domains"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
catalog_id = response_as_json[0]["urn"]
catalog_id

'urn:li:domain:medicamentos'

In [86]:
url = data_space_provider + "/api/v1/datahub/domains/{}/datasets".format(catalog_id)
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
dataset_id = response_as_json[0]["urn"]
dataset_id

'urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)'

## Create policies

In [87]:
url = data_space_provider + "/api/v1/datasets/{}/policies".format(dataset_id)
payload = {
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }

    ],
    "obligation": [],
    "prohibition": []
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["@id"]
target_id = response_as_json["target"]
print(json.dumps(response_as_json, indent=4))

{
    "@id": "urn:uuid:2b5fb53a-fd11-4c57-97e0-ff40092dd06e",
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }
    ],
    "obligation": [],
    "@type": "Offer",
    "prohibition": [],
    "target": "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)"
}


## Contract Negotiation

In [88]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
  "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
  "offer": {
    "@id": "urn:uuid:2b5fb53a-fd11-4c57-97e0-ff40092dd06e",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
    "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
    "state": "REQUESTED"
  }
}


In [89]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id, # remove to test offer from provider
    "providerPid": cn_provider_id, # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
  "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
  "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
  "offer": {
    "@id": "urn:uuid:2b5fb53a-fd11-4c57-97e0-ff40092dd06e",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
    "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
    "state": "OFFERED"
  }
}


In [90]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
  "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
  "offer": {
    "@id": "urn:uuid:2b5fb53a-fd11-4c57-97e0-ff40092dd06e",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
    "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
    "state": "REQUESTED"
  }
}


In [91]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
  "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
  "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
  "offer": {
    "@id": "urn:uuid:2b5fb53a-fd11-4c57-97e0-ff40092dd06e",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
    "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
    "state": "OFFERED"
  }
}


In [92]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-acceptance"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
  "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
    "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
    "state": "ACCEPTED"
  }
}


In [93]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-agreement"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
agreement_id=response_as_json["agreement"]["@id"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
  "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
  "agreement": {
    "@id": "urn:uuid:f01cb9f4-106e-4785-80bd-0765758aba34",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Agreement",
    "target": "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)",
    "assigner": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0",
    "assignee": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
    "timestamp": "2025-07-15T10:51:38.187896+00:00"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "p

In [94]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-verification"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
  "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
    "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
    "state": "VERIFIED"
  }
}


In [95]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-finalization"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
  "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:1bfde270-556f-4eaf-a1c7-da5569be802a",
    "consumerPid": "urn:uuid:5d8d92a2-ff24-47a5-9044-84555c13bcac",
    "state": "FINALIZED"
  }
}


## Transfer Negotiation


In [106]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id, # to string, instead of urn...
    "agreementId": agreement_id,
    "format": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_consumer_id = response_as_json["consumerPid"]
transfer_provider_id = response_as_json["providerPid"]
callback_address = response_as_json["callbackAddress"]
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
  "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
  "agreementId": "urn:uuid:f01cb9f4-106e-4785-80bd-0765758aba34",
  "callbackAddress": "http://127.0.0.1:1100/urn:uuid:e0d311c6-24ae-4e2c-96ef-89c4ecd91e6b",
  "format": "http+pull",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
    "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
    "state": "REQUESTED"
  }
}


In [107]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-start"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
  "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
  "dataAddress": {
    "@type": "DataAddress",
    "endpointType": "http",
    "endpoint": "http://127.0.0.1:1200/data/urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
    "endpointProperties": [
      {
        "@type": "EndpointProperty",
        "name": "authType",
        "value": ""
      },
      {
        "@type": "EndpointProperty",
        "name": "authorization",
        "value": ""
      }
    ]
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
    "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
    "state": "STARTED"
  }
}


In [108]:
url = data_space_consumer + "/api/v1/dataplane/" + transfer_consumer_id
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
consumer_data_address = response_as_json["process_address"]["url"]
print(json.dumps(response_as_json, indent=2))

{
  "id": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
  "process_direction": "PULL",
  "upstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "downstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "process_address": {
    "protocol": "http",
    "url": "http://127.0.0.1:1100/data/urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
    "auth_type": "",
    "auth_content": ""
  },
  "created_at": "2025-07-15T10:59:42.365098",
  "updated_at": "2025-07-15T10:59:42.416214",
  "state": "STARTED"
}


In [109]:
try :
    response = requests.request("GET", consumer_data_address, headers=json_header)
    print(response.json())
except HTTPError as e:
    print(e)


{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.', 'terms': 'https://open.fda.gov/terms/', 'license': 'https://open.fda.gov/license/', 'last_updated': '2025-04-28', 'results': {'skip': 0, 'limit': 1, 'total': 443884}}, 'results': [{'safetyreportversion': '3', 'safetyreportid': '10003321', 'primarysourcecountry': 'JP', 'occurcountry': 'JP', 'transmissiondateformat': '102', 'transmissiondate': '20151125', 'reporttype': '2', 'serious': '1', 'seriousnesshospitalization': '1', 'seriousnessother': '1', 'receivedateformat': '102', 'receivedate': '20140312', 'receiptdateformat': '102', 'receiptdate': '20150818', 'fulfillexpeditecriteria': '1', 'companynumb': 'JP-GLAXOSMITHKLINE-B0975691A', 'duplicate': '1', 'reportduplicate': {'duplicatesource': 'GLAXOSMITH

In [110]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
  "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
    "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
    "state": "SUSPENDED"
  }
}


In [111]:
try :
    response = requests.request("GET", consumer_data_address, headers=json_header)
    if response.status_code == 403 :
        raise HTTPError(consumer_data_address, "403 Forbidden", "403 Forbidden", "application/json", response)
    print(response.json())
except HTTPError as e:
    print(e)

HTTP Error 403 Forbidden: 403 Forbidden


In [112]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-start"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
  "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
    "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
    "state": "STARTED"
  }
}


In [113]:
try :
    response = requests.request("GET", consumer_data_address, headers=json_header)
    print(response.json())
except HTTPError as e:
    print(e)

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.', 'terms': 'https://open.fda.gov/terms/', 'license': 'https://open.fda.gov/license/', 'last_updated': '2025-04-28', 'results': {'skip': 0, 'limit': 1, 'total': 443884}}, 'results': [{'safetyreportversion': '3', 'safetyreportid': '10003321', 'primarysourcecountry': 'JP', 'occurcountry': 'JP', 'transmissiondateformat': '102', 'transmissiondate': '20151125', 'reporttype': '2', 'serious': '1', 'seriousnesshospitalization': '1', 'seriousnessother': '1', 'receivedateformat': '102', 'receivedate': '20140312', 'receiptdateformat': '102', 'receiptdate': '20150818', 'fulfillexpeditecriteria': '1', 'companynumb': 'JP-GLAXOSMITHKLINE-B0975691A', 'duplicate': '1', 'reportduplicate': {'duplicatesource': 'GLAXOSMITH

In [114]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
  "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
    "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
    "state": "SUSPENDED"
  }
}


In [115]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-completion"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
  "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:3c525208-bfbf-4219-938b-d9e7d5916dc1",
    "consumerPid": "urn:uuid:abfaf17d-b976-40ab-9ac4-4f2e12395ef6",
    "state": "COMPLETED"
  }
}
